In [1]:
import cv2
import mediapipe as mp



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\erwan\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\erwan\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\erwan\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\erwan\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:

# Inisialisasi MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Konfigurasi untuk menggambar (warna dan ketebalan)
drawing_spec_face = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
drawing_spec_hands = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

# Mulai menangkap video dari webcam
cap = cv2.VideoCapture(0)

# Inisialisasi model Holistic dari MediaPipe
# min_detection_confidence: seberapa yakin model harus mendeteksi
# min_tracking_confidence: seberapa yakin model harus melacak
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        # Baca frame dari webcam
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Balik gambar secara horizontal agar seperti cermin
        image = cv2.flip(image, 1)

        # Konversi warna BGR ke RGB karena MediaPipe menggunakan RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Proses gambar dengan model Holistic untuk deteksi
        results = holistic.process(image_rgb)

        # Gambar kembali landmark ke gambar asli (BGR)

        # 1. Gambar Face Mesh (jaring-jaring wajah)
        # Menggunakan FACEMESH_TESSELATION untuk menggambar garis-garis jaring
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=results.face_landmarks,
            connections=mp_holistic.FACEMESH_TESSELATION,
            landmark_drawing_spec=None, # Tidak menggambar titik landmark
            connection_drawing_spec=drawing_spec_face
        )

        # 2. Gambar Kerangka Tangan Kiri
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=results.left_hand_landmarks,
            connections=mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=drawing_spec_hands,
            connection_drawing_spec=drawing_spec_hands
        )

        # 3. Gambar Kerangka Tangan Kanan
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=results.right_hand_landmarks,
            connections=mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=drawing_spec_hands,
            connection_drawing_spec=drawing_spec_hands
        )

        # Tampilkan hasilnya
        cv2.imshow('MediaPipe Face and Hand Detection', image)

        # Keluar dari loop jika tombol 'q' ditekan
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Lepaskan webcam dan tutup semua jendela
cap.release()
cv2.destroyAllWindows()